In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
def get_connections(url):

  r=requests.get(url)
  soup=BeautifulSoup(r.text)

  nodes=[]
  errors=[]
  for line in soup.find_all('a'):
    if 'href="/wiki/' in str(line):
      try:
        link=str(line).split('href="')[1].split('"')[0]
        title=str(line).split('title="')[1].split('"')[0]
        if ':' not in link:
          link='https://en.wikipedia.org'+link
          nodes.append([link,title])
        else:
          pass
      except:
        errors.append(line)

    df=pd.DataFrame(nodes,columns=['link','title']).drop_duplicates()
    links=df['link'].tolist()
    titles=df['title'].tolist()

  return links,titles,errors

In [3]:
links,titles,_=get_connections('https://en.wikipedia.org/wiki/PageRank')

In [4]:
connection_df=pd.DataFrame(np.zeros((len(links),len(links))),index=links,columns=links)

In [5]:
columns=connection_df.columns.tolist()
for link,title in tqdm(zip(links,titles)):
  connection_links,connection_titles,_=get_connections(link)
  for connection_link,connection_title in zip(connection_links,connection_titles):
    if connection_link in columns:
      connection_df.at[link,connection_link]=1

136it [03:43,  1.64s/it]


In [6]:
connection_df

,https://en.wikipedia.org/wiki/Main_Page,https://en.wikipedia.org/wiki/PageRank,https://en.wikipedia.org/wiki/Algorithm,https://en.wikipedia.org/wiki/Google_Search,https://en.wikipedia.org/wiki/Webpages,https://en.wikipedia.org/wiki/Search_engine,https://en.wikipedia.org/wiki/Larry_Page,https://en.wikipedia.org/wiki/Network_theory#Link_analysis,https://en.wikipedia.org/wiki/Weighting,https://en.wikipedia.org/wiki/Hyperlink,...,https://en.wikipedia.org/wiki/Wayback_Machine,https://en.wikipedia.org/wiki/Forbes,https://en.wikipedia.org/wiki/ISBN_(identifier),https://en.wikipedia.org/wiki/Bibcode_(identifier),https://en.wikipedia.org/wiki/PMID_(identifier),https://en.wikipedia.org/wiki/Barry_Schwartz_(technologist),https://en.wikipedia.org/wiki/Search_Engine_Land,https://en.wikipedia.org/wiki/PMC_(identifier),https://en.wikipedia.org/wiki/Lesley_Ward,https://en.wikipedia.org/wiki/Amy_Langville
https://en.wikipedia.org/wiki/Main_Page,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://en.wikipedia.org/wiki/PageRank,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
https://en.wikipedia.org/wiki/Algorithm,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
https://en.wikipedia.org/wiki/Google_Search,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
https://en.wikipedia.org/wiki/Webpages,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://en.wikipedia.org/wiki/Barry_Schwartz_(technologist),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
https://en.wikipedia.org/wiki/Search_Engine_Land,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://en.wikipedia.org/wiki/PMC_(identifier),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
https://en.wikipedia.org/wiki/Lesley_Ward,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
connection_df.drop('https://en.wikipedia.org/wiki/Main_Page',inplace=True)
connection_df=connection_df[connection_df.columns.tolist()[1:]]

In [8]:
connection_df.to_excel('connection_matrix.xlsx')

In [27]:
M1=connection_df.to_numpy()

for i,row in enumerate(M1):
  if np.sum(row)==0:
    M1[i]=1/M1.shape[0]
  else:
    M1[i]/=np.sum(row)

M1*=0.9
M1=np.transpose(M1)

M2=np.ones(M1.shape)/M1.shape[0]

M=M1+M2

In [28]:
M

array([[0.01407407, 0.00740741, 0.03048433, ..., 0.00740741, 0.00740741,
        0.15740741],
       [0.01407407, 0.07663818, 0.03048433, ..., 0.00740741, 0.00740741,
        0.00740741],
       [0.01407407, 0.00740741, 0.03048433, ..., 0.00740741, 0.00740741,
        0.00740741],
       ...,
       [0.01407407, 0.00740741, 0.00740741, ..., 0.11990741, 0.00740741,
        0.00740741],
       [0.01407407, 0.00740741, 0.00740741, ..., 0.00740741, 0.23240741,
        0.00740741],
       [0.01407407, 0.00740741, 0.00740741, ..., 0.00740741, 0.00740741,
        0.15740741]])

In [29]:
from scipy.linalg import null_space

In [30]:
eigenvec=null_space((M-np.eye(M.shape[0])))

In [31]:
eigenvec

array([], shape=(135, 0), dtype=float64)